In [ ]:
tfrecord_files_train,tfrecord_files_val=dataset_builder.get_tfrecord_filenames()
print("Found the following TFrecords:\n","\n".join(tfrecord_files_train),"\n", "\n".join(tfrecord_files_val))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train,model.place_training_labels)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_val,model.place_training_labels)
print("Validation dataset shape:",dsv)

In [ ]:
import datetime
import dataset_functions
import models.six_stage_linear_model as model
import callbacks
import dataset_builder
import load_weights
import loss_metrics
from utils import now

In [ ]:
import config as c